Post-suite2P data preprocessing for communication subspace analysis

To use the Semedo et al. code for reduced rank regression etc., the output should be an nxm matrix, where n=number of cells and m = (n trials x n samples in trial bin). If you save this as a .npy, it can be imported to MATLAB using the readNPY.m function found in [npy-matlab](https://github.com/kwikteam/npy-matlab) package.

0.7 neuropil subtraction applied: this can be changed in utils_funcs.s2p_loader

In [1]:
#ipython magic
%matplotlib inline 
%load_ext autoreload
%autoreload 2

In [2]:
# general imports
import sys
sys.path.append('/home/thijs/Google Drive/repos/Vape/utils')
sys.path.append('/home/thijs/Google Drive/repos/Vape/jupyter/subspace')
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import utils_funcs as utils
from paq2py import paq_read
import sfuncs
import pickle
import h5py

In [3]:
# process 1 plane at a time for now
plane = 1 # select plane to process. This is zero-indexed!
tot_planes = 2 # how many planes did you record in total?
fs = 1/0.043478004 # total frame rate Hz (1/frame period from xml file accompanying your recording. This will change when using custom ROIs)
fs = fs / tot_planes #total frame rate is divided by number of planes
print('plane recorded at {0:.4f} frames per second'.format(fs))

cwd = os.getcwd()
os.chdir('/home/thijs/Google Drive/oxford/packerlab/data_sarah')  # move to data folder

raw = np.load('raw.npy')
spks = np.load('spks.npy')

## To load the stat and paq files, the np.load() functions needs to be altered:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
stat = np.load('stat.npy')  # stat is a np array (of len n_neurons), each el is a dict
paq = np.load('paq.npy')
paq = paq.item()  # unpack to dictionary
# restore np.load for future normal usage
np.load = np_load_old

dff = utils.dfof2(raw) #compute dF/F

plane recorded at 11.5001 frames per second


## Split cells into S1 and S2

In [4]:
# In suite2p, find the cell that is furthest to the right in area S1 and note its index. 
# We will split the cells by a linear plane through the cell's X-coordinate
c = sfuncs.getKeyFromStat_1plane(stat,'med',conditional=('original_index',19)) 
print('X coordinate at S1S2 boundary = {}'.format(c[0][1]))

#split cells by boundary coordinate
boundary = c[0][1]
S1_indices, S2_indices = sfuncs.separateByX(stat, boundary) #split all cells in stat based on X coordinate
print('number S1 cells =',len(S1_indices),'; number S2 cells =',len(S2_indices))

#check the split has been performed correctly
assert (len(S2_indices) + len (S1_indices) == stat.shape[0]) # should be True

#check there are no cells in the same group (output should be empty)
assert len(set(S1_indices) & set(S2_indices) ) == 0

Retrieving values for med key where condition met: original_index == 19
X coordinate at S1S2 boundary = 458.0
number S1 cells = 185 ; number S2 cells = 157


In [5]:
# Use indices to split data by region
S1_stat = stat[S1_indices]
S2_stat = stat[S2_indices]
#get deconvolved spikes and dF/F for each area
S1_spks = spks[S1_indices, :]
S2_spks = spks[S2_indices, :]
S1_dff = dff[S1_indices, :]
S2_dff = dff[S2_indices, :]
assert S1_dff.shape[1] == S2_dff.shape[1]
print(f'Number of neurons in S1: {S1_dff.shape[0]}, and S2: {S2_dff.shape[0]}. Number of time points {S1_dff.shape[1]}')

Number of neurons in S1: 185, and S2: 157. Number of time points 12881


## Split data by trial

In [6]:
tot_frames = dff.shape[1] * tot_planes

stim_start = utils.paq_data(paq, 'piezo_stim', threshold_ttl = True)
stim_start_frames = sfuncs.stim_start_frame(paq,'piezo_stim')

frame_clock = utils.paq_data(paq, 'frame_clock', threshold_ttl = True)
frame_clock = frame_clock[:tot_frames] # get rid of foxy bonus frames
frame_clock = frame_clock[plane::tot_planes] # just take clocks from the frame you care about

In [7]:
# set the time window for each trial bin here. The authors of the Semedo paper take 1 second of data 
# per trial. As our stim is longer i'll try taking 2 seconds post-stimulus onset. 
# Worth playing with this

pre_seconds = 0
post_seconds = 2
pre_frames = int(pre_seconds * fs) # number of frames before stim onset to inlcude in the trial
post_frames = int(post_seconds * fs) # number of frames after stim onset to inlcude in the trial

offset_s = 0 # option to offset trial bins from stimulus onset
second_in_samples = 22000
offset_frames = int(offset_s * second_in_samples)

In [8]:
#spikes S1
spks_trials_S1 = utils.flu_splitter(S1_spks, frame_clock, stim_start, pre_frames, post_frames)
spks_trials_S1 = spks_trials_S1[0]
print(f'Shape spikes trial-based S1 {spks_trials_S1.shape}')

#spikes S2
spks_trials_S2 = utils.flu_splitter(S2_spks, frame_clock, stim_start, pre_frames, post_frames)
spks_trials_S2 = spks_trials_S2[0]
print(f'Shape spikes trial-based S2 {spks_trials_S2.shape}')

#dFF S1
dff_trials_S1 = utils.flu_splitter(S1_dff, frame_clock, stim_start+offset_frames, pre_frames, post_frames)
dff_trials_S1 = dff_trials_S1[0]
assert dff_trials_S1.shape == spks_trials_S1.shape

#dFF S2
dff_trials_S2 = utils.flu_splitter(S2_dff, frame_clock, stim_start+offset_frames, pre_frames, post_frames)
dff_trials_S2 = dff_trials_S2[0]
assert dff_trials_S2.shape == spks_trials_S2.shape

Shape spikes trial-based S1 (185, 23, 100)
Shape spikes trial-based S2 (157, 23, 100)


## Subtract mean and reshape back to 2D


In [9]:
def zero_mean(data):
    """Assume 3D, last dimension is trials"""
    assert len(data.shape) == 3
    mean_data_reshaped = np.mean(data, 2)[:, :, np.newaxis]  # mean over trials
    data_subbed = data - mean_data_reshaped  # zero mean data
    assert data_subbed.shape == data.shape
    data_subbed_rs = np.reshape(a=data_subbed, 
                                newshape=(data_subbed.shape[0], 
                                          data_subbed.shape[1] * data_subbed.shape[2]),
                                order='F')  # reshape, take into account right order
    return data_subbed_rs

## Export data to use

In [10]:
## Define empty dict
dict_export = {'dff_s1': np.array([]), 'dff_s2': np.array([]), 
               'spikes_s1': np.array([]), 'spikes_s2': np.array([])}

In [11]:
## Fill dict with the zero-meaned data
dict_export['dff_s1'] = zero_mean(dff_trials_S1)
dict_export['dff_s2'] = zero_mean(dff_trials_S2)
dict_export['spikes_s1'] = zero_mean(spks_trials_S1)
dict_export['spikes_s2'] = zero_mean(spks_trials_S2)


In [12]:
## Save as h5 file so it can be read in matlab/pyton
print(f'Current directory: {os.getcwd()}')
hfile = h5py.File('data_sarah_s1s2_zero_mean.h5', 'w') # create file
for key, val in dict_export.items():  # store data
    hfile.create_dataset(key, data=val)
hfile.close()  # close file

Current directory: /home/thijs/Google Drive/oxford/packerlab/data_sarah


In [ ]:
## To load:

# hfile = h5py.File('data_s1s2_zero_mean.h5', 'r')  # open with read access
# data = hfile['dff_s1']  # memory-mapped access (until hfile is closed)
# data = hfile['dff_s1'].value  # RAM access (permanent)
# hfile.close()